In [1]:
%run GPT2.ipynb

torch version: 2.1.2+cpu
tiktoken version: 0.5.2
Input text: 
 Every effort moves you 
 Every day holds a
Input batch:
 tensor([[6109, 3626, 6100,  345],
        [6109, 1110, 6622,  257]])

Output shape: torch.Size([2, 4, 50257])
Total number of parameters: 163,009,536
Token embedding layer shape: torch.Size([50257, 768])
Output layer shape: torch.Size([50257, 768])
Number of trainable parameters considering weight tying: 124,412,160
Total size of the model: 621.83 MB
the input text >>> Hello, I am
encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])
Output: tensor([[15496,    11,   314,   716, 27018, 24086, 47843, 30961, 42348,  7267]])
Output length: 10
after decoding then get >>> Hello, I am Featureiman Byeswickattribute argue


In [2]:
@dataclass
class Params:
    vocab_size= 50257   # Vocabulary size
    context_length= 256 # Context length
    emb_dim= 768         # Embedding dimension
    n_heads= 12          # Number of attention heads
    n_layers= 12         # Number of layers
    drop_rate= 0.1       # Dropout rate
    qkv_bias= False       # Query-Key-Value bias

my_params = Params()

In [ ]:
torch.manual_seed(123)
model = GPTModel(my_params)
model.eval()  # Disable dropout during inference

## DOWNLOADING & PREPARING DATA

In [3]:

import os
import urllib.request

file_path = r"./data/the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
    with urllib.request.urlopen(url) as response:
        text_data = response.read().decode('utf-8')
    with open(file_path, "w", encoding="utf-8") as file:
        file.write(text_data)
else:
    with open(file_path, "r", encoding="utf-8") as file:
        text_data = file.read()

# First 100 characters
print(text_data[:99])
# Last 100 characters
print(text_data[-99:])

total_characters = len(text_data)
tokenizer = tiktoken.get_encoding("gpt2")
total_tokens = len(tokenizer.encode(text_data))

print("Characters:", total_characters)
print("Tokens:", total_tokens)

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."
Characters: 20479
Tokens: 5145


In [4]:
from utils import dataloaders

# Train/validation ratio
train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)

train_loader = dataloaders.create_dataloader_v1(
    train_data,
    batch_size=2,
    max_length=my_params.context_length, #GPT_CONFIG_124M["context_length"],
    stride=my_params.context_length,
    drop_last=True,
    shuffle=True,
    num_workers=0
)

val_loader = dataloaders.create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=my_params.context_length, #GPT_CONFIG_124M["context_length"],
    stride=my_params.context_length,
    drop_last=False,
    shuffle=False,
    num_workers=0
)

# Sanity check
if total_tokens * (train_ratio) < my_params.context_length:
    print("Not enough tokens for the training loader. "
          "Try to lower the `my_params.context_length` or "
          "increase the `training_ratio`")

if total_tokens * (1-train_ratio) < my_params.context_length:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `my_params.context_length` or "
          "decrease the `training_ratio`")

In [5]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


In [6]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4608
Validation tokens: 512
All tokens: 5120


## TEST METRICS CALC

In [ ]:
torch.manual_seed(123) # For reproducibility due to the shuffling in the data loader

def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(
        logits.flatten(0, 1), 
        target_batch.flatten()
    )
    return loss


def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0.
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))

    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break

    return total_loss / num_batches

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, model, device)
    val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

In [11]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                       eval_freq, eval_iter, start_context, tokenizer):
    # Initialize lists to track losses and tokens seen
    train_losses, val_losses, track_tokens_seen = [], [], []
    tokens_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode
        
        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            tokens_seen += input_batch.numel()
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                track_tokens_seen.append(tokens_seen)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Print a sample text after each epoch
        generate_and_print_sample(
            model, tokenizer, device, start_context
        )

    return train_losses, val_losses, track_tokens_seen


def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    flat = token_ids.squeeze(0) # remove batch dimension
    return tokenizer.decode(flat.tolist())

def generate_and_print_sample(model, tokenizer, device, start_context):
    model.eval()
    context_size = model.pos_emb.weight.shape[0]
    encoded = text_to_token_ids(start_context, tokenizer).to(device)
    with torch.no_grad():
        token_ids = generate_text_simple(
            model=model, idx=encoded,
            max_new_tokens=50, context_size=context_size
        )
        decoded_text = token_ids_to_text(token_ids, tokenizer)
        print(decoded_text.replace("\n", " "))  # Compact print format
    model.train()

In [ ]:
torch.manual_seed(123)

model = GPTModel(my_params)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

## USING PYTORCH Lighting

In [7]:
import os
from torch import optim, nn, utils, Tensor
# from torchvision.datasets import MNIST
# from torchvision.transforms import ToTensor
try:
    import lightning as pl
except ImportError:
    !pip install lightning
    import lightning as pl


from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import Callback
logger = TensorBoardLogger(save_dir=r"./")

# %reload_ext tensorboard
# %tensorboard --logdir=lightning_logs/
# trainer = Trainer()
# class MetricsLogger(Callback):
#     def __init__(self):
#         super().__init__()
#         self.train_losses = []
#         self.val_losses = []

#     def on_train_batch_end(self, trainer, pl_module, outputs, batch, batch_idx, dataloader_idx):
#         self.train_losses.append(outputs['train_loss'].item())

#     def on_validation_end(self, trainer, pl_module):
#         metrics = trainer.callback_metrics
#         self.val_losses.append(metrics['val_loss'].item())

# # Add the callback to the trainer
# metrics_logger = MetricsLogger()

In [19]:
# define any number of nn.Modules (or use your current ones)
# mygpt = GPTModel(my_params)
torch.manual_seed(123)

# define the LightningModule
class MyGPT(pl.LightningModule):
    def __init__(self, params):
        super().__init__()
        self.save_hyperparameters()
        self.model = GPTModel(params)

    def forward(self, in_idx):
        return self.model(in_idx)

    def training_step(self, batch, batch_idx):
        # training_step defines the train loop.
        # it is independent of forward
        input_batch, target_batch = batch
        logits = self.model(input_batch)
        loss = nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        # Logging to TensorBoard (if installed) by default
        self.log("train_loss", loss, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        # this is the test loop
        input_batch, target_batch = batch
        logits = self.model(input_batch)
        loss = nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
        self.log("val_loss", loss, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)
        return optimizer

    # def any_lightning_module_function_or_hook(self):
    #     tensorboard_logger = self.logger.experiment
    #     fake_images = torch.Tensor(32, 3, 28, 28)
    #     tensorboard_logger.add_image("generated_images", fake_images, 0)

# init the model
model = MyGPT(my_params)

# train the model (hint: here are some helpful Trainer arguments for rapid idea iteration)
trainer = pl.Trainer(limit_train_batches=100, max_epochs=10, val_check_interval=5, logger=logger, log_every_n_steps=5)# , check_val_every_n_epoch=1
trainer.fit(model=model, train_dataloaders=train_loader, val_dataloaders=val_loader) 

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
c:\Users\Hugo Xue\.conda\envs\myllm\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:652: Checkpoint directory ./lightning_logs\version_6\checkpoints exists and is not empty.

  | Name  | Type     | Params | Mode 
-------------------------------------------
0 | model | GPTModel | 162 M  | train
-------------------------------------------
162 M     Trainable params
0         Non-trainable params
162 M     Total params
649.679   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\Hugo Xue\.conda\envs\myllm\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
c:\Users\Hugo Xue\.conda\envs\myllm\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:424: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [ ]:
# # Plotting the metrics
# plt.figure(figsize=(12, 5))
# plt.subplot(1, 2, 1)
# plt.plot(metrics_logger.train_losses, label='Train Loss')
# plt.plot(metrics_logger.val_losses, label='Validation Loss')
# plt.xlabel('Batch')
# plt.ylabel('Loss')
# plt.legend()

# plt.subplot(1, 2, 2)
# plt.plot(metrics_logger.val_accs, label='Validation Accuracy')
# plt.xlabel('Epoch')
# plt.ylabel('Accuracy')
# plt.legend()

# plt.show()

In [20]:
model.eval()

tokenizer = tiktoken.get_encoding("gpt2")

# with torch.no_grad():
token_ids = generate_text_simple(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=25,
    context_size=my_params.context_length
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you know," was one of the picture for nothing--I had a little of a flash that he had been of the fact,


In [21]:
def generate(model, idx, max_new_tokens, context_size, temperature=0.0, top_k=None, eos_id=None):

    # For-loop is the same as before: Get logits, and only focus on last time step
    for _ in range(max_new_tokens):
        idx_cond = idx[:, -context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:, -1, :]

        # New: Filter logits with top_k sampling
        if top_k is not None:
            # Keep only top_k values
            top_logits, _ = torch.topk(logits, top_k)
            min_val = top_logits[:, -1]
            logits = torch.where(logits < min_val, torch.tensor(float('-inf')).to(logits.device), logits)

        # New: Apply temperature scaling
        if temperature > 0.0:
            logits = logits / temperature

            # Apply softmax to get probabilities
            probs = torch.softmax(logits, dim=-1)  # (batch_size, context_len)

            # Sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (batch_size, 1)

        # Otherwise same as before: get idx of the vocab entry with the highest logits value
        else:
            idx_next = torch.argmax(logits, dim=-1, keepdim=True)  # (batch_size, 1)

        if idx_next == eos_id:  # Stop generating early if end-of-sequence token is encountered and eos_id is specified
            break

        # Same as before: append sampled index to the running sequence
        idx = torch.cat((idx, idx_next), dim=1)  # (batch_size, num_tokens+1)

    return idx

torch.manual_seed(123)

token_ids = generate(
    model=model,
    idx=text_to_token_ids("Every effort moves you", tokenizer),
    max_new_tokens=15,
    context_size=my_params.context_length,
    top_k=25,
    temperature=1.4
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you know I found my a good _ himself it was such a laugh that there
